# Deep Galerkin Method (DGM)
### S1 = sigma(w1*x + b1)  Z(l) = sigma(u*x + w*S + b) l=1,...,L  G(l) = sigma(u*x + w*S + b) l=1,...,L  
### R(l) = sigma(u*x + w*S + b) l=1,...,L   H(l) = sigma(u*x + w*(S Hadamard R) + b)  l=1,...,L  
### S(L+1) = (1-G) Hadamard H + Z Hadamard S  f = w*S(L+1) + b  

In [1]:
#import needed packages
import tensorflow as tf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

/home/wangyaohong/.conda/envs/tensorflow1.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wangyaohong/.conda/envs/tensorflow1.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wangyaohong/.conda/envs/tensorflow1.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wangyaohong/.

**input_dim:** dimensionality of imput data  
**output_dim:** number of output for LSTM layer  
**trans1, trans2 (str):** activation functions used inside the layer;  
one of: "tanh"(default), "relu" or "sigmoid"  
**u vectors:** weighting vectors for inputs original inputs x  
**w vectors:** weighting vectors for output of previous layer  
**S:** output of previous layer  
**X:** data input

In [2]:
# LSTM-like layer used in DGM
class LSTMLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, trans1 = "tanh", trans2 = "tanh"):
        
        super(LSTMLayer, self).__init__()
        
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        if trans1 == "tanh":
            self.trans1 = tf.nn.tanh
        elif trans1 == "relu":
            self.trans1 = tf.nn.relu
        elif trans1 == "sigmoid":
            self.trans1 = tf.nn.sigmoid
        
        if trans2 == "tanh":
            self.trans2 = tf.nn.tanh
        elif trans2 == "relu":
            self.trans2 = tf.nn.relu
        elif trans2 == "sigmoid":
            self.trans2 = tf.nn.relu
        
        # u vectors (weighting vectors for inputs original inputs x)
        self.Uz = self.add_variable("Uz", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Ug = self.add_variable("Ug", shape=[self.input_dim ,self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Ur = self.add_variable("Ur", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Uh = self.add_variable("Uh", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        
        # w vectors (weighting vectors for output of previous layer)        
        self.Wz = self.add_variable("Wz", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wg = self.add_variable("Wg", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wr = self.add_variable("Wr", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wh = self.add_variable("Wh", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        
        # bias vectors
        self.bz = self.add_variable("bz", shape=[1, self.output_dim])
        self.bg = self.add_variable("bg", shape=[1, self.output_dim])
        self.br = self.add_variable("br", shape=[1, self.output_dim])
        self.bh = self.add_variable("bh", shape=[1, self.output_dim])
    
    
    def call(self, S, X):

        Z = self.trans1(tf.add(tf.add(tf.matmul(X,self.Uz), tf.matmul(S,self.Wz)), self.bz))
        G = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ug), tf.matmul(S, self.Wg)), self.bg))
        R = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ur), tf.matmul(S, self.Wr)), self.br))
        
        H = self.trans2(tf.add(tf.add(tf.matmul(X,self.Uh), tf.matmul(tf.multiply(S, R), self.Wh)), self.bh))
        
        S_new = tf.add(tf.multiply(tf.subtract(tf.ones_like(G), G), H), tf.multiply(Z,S))
        
        return S_new

**input_dim:** dimensionality of input data  
**output_dim:** number of outputs for dense layer  
**transformation:** activation function used inside the layer; using None is equivalent to the identity map  
**w vectors:** weighting vectors for output of previous layer  
**X:** input to layer  

In [3]:
# Fully connected layer(dense)
class DenseLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, transformation=None):
        
        super(DenseLayer,self).__init__()
        self.output_dim = output_dim
        self.input_dim = input_dim

        self.W = self.add_variable("W", shape=[self.input_dim, self.output_dim],
                                   initializer = tf.contrib.layers.xavier_initializer())
        
        # bias vectors
        self.b = self.add_variable("b", shape=[1, self.output_dim])
        
        if transformation:
            if transformation == "tanh":
                self.transformation = tf.tanh
            elif transformation == "relu":
                self.transformation = tf.nn.relu
        else:
            self.transformation = transformation
    
    
    def call(self,X):
        
        S = tf.add(tf.matmul(X, self.W), self.b)
                
        if self.transformation:
            S = self.transformation(S)
        
        return S

**n_layers:** number of intermediate LSTM layers  
**input_dim:** spaital dimension of input data (excludes time dimension)  
**final_trans:** transformation used in final layer
define initial layer as fully connected
to account for time inputs we use input_dim+1 as the input dimension
**t:** sampled time inputs  
**x:** sampled space inputs  
Run the DGM model and obtain fitted function value at the inputs (t,x)

In [4]:
# Neural network architecture used in DGM

class DGMNet(tf.keras.Model):
    
    def __init__(self, layer_width, n_layers, input_dim, final_trans=None):
        
        super(DGMNet,self).__init__()
        
        self.initial_layer = DenseLayer(layer_width, input_dim+1, transformation = "tanh")
        
        self.n_layers = n_layers
        self.LSTMLayerList = []
                
        for _ in range(self.n_layers):
            self.LSTMLayerList.append(LSTMLayer(layer_width, input_dim+1,trans1 = "tanh", trans2 = "tanh"))
        
        self.final_layer = DenseLayer(1, layer_width, transformation = final_trans)
    
    def call(self,t,x):

        X = tf.concat([t,x],1)
        
        # call initial layer
        S = self.initial_layer.call(X)
        
        # call intermediate LSTM layers
        for i in range(self.n_layers):
            S = self.LSTMLayerList[i].call(S,X)
        
        # call final LSTM layers
        result = self.final_layer.call(S)
        
        return result

Parameters

In [5]:
# OU process parameters（Ornstein-Uhlenbeck Process）
kappa = 0
theta = 0.5
sigma = 2

# mean and standard deviation for (normally distributed) process starting value
alpha = 0.0
beta = 1

# tenminal time
T = 1.0

# bounds of sampling region for space dimension, i.e. sampling will be done on
# [multipliter*Xlow, multiplier*Xhigh]
Xlow = -4.0
Xhigh = 4.0
x_multiplier = 2.0
t_multiplier = 1.5

# neural network parameters
num_layers = 3
nodes_per_layer = 50
learning_rate = 0.001

# Training parameters
sampling_stages = 500
steps_per_sample = 10

# Sampling parameters
nSim_t = 5
nSim_x_interior = 50
nSim_x_initial = 50

# Save options
saveName = 'FokkerPlanck'
saveFigure = False

**OU Simulation function(Ornstein-Uhlenbeck Process)**  
Simulate end point of Ornstein-Uhlenbeck process with normally distributed random starting value  
**alpha:** mean of random starting value  
**beta:** standard deviation of random starting value   
**theta:** mean reversion level    
**kappa:** mean reversion rate  
**sigma:** volatility  
**nSim:** number of simulations  
**T:** terminal time  

In [6]:
def simulateOU_GaussianStart(alpha, beta, theta, kappa, sigma, nSim, T):
    
    # simulate initial point based on normal distribution
    X0 = np.random.normal(loc = alpha, scale = beta, size = nSim)
    
    # mean and variance of OU endpoint
    m = theta + (X0 - theta) * np.exp(-kappa * T)
    v = np.sqrt(sigma**2 / (2 * kappa) * (1 - np.exp(-2*kappa*T)))
    
    # simulate endpoint
    Xt = np.random.normal(m,v)    
    
    return Xt

Sample time-space points from the function's domain;  
point are sampled uniformly on the interior of the domain, at the initial/terminal time points  
and along the spatial boundary at different time points.  
**nSim_t:** number of (interior) time points to sample  
**nSim_x_interior:** number of space points in the interior of the function's domain to sample  
**nSim_x_initial:** number of space points at initial time to sample (initial condition)

In [7]:
# Sampling function - random sample time-space pairs
def sampler(nSim_t, nSim_x_interior, nSim_x_initial):
    
    # Sampler1: domain interior
    t = np.random.uniform(low=0, high=T*t_multiplier, size=[nSim_t, 1])
    x_interior = np.random.uniform(low=Xlow*x_multiplier, high=Xhigh*x_multiplier, size=[nSim_x_interior, 1])
    
    # Sampler: spatial boundary
    # no spatial boundary condition for this problem 
    
    # Sampler3: initial/terminal condition
    x_initial = np.random.uniform(low=Xlow*1.5, high=Xhigh*1.5, size = [nSim_x_initial, 1])
    
    return t, x_interior, x_initial

Compute total loss for training.     
The loss is based o the PDE satisfied by the negative-exponential of the density and NOT the density   
itself, i.e. the u(t,x) in p(t,x) = exp(-u(t,x)) / c(t) where p is the density and c is the normalization constant.    
**model:** DGM model object   
**t:** sampled (interior) time points  
**x_interior:** sampled space points in the interior of the function's domain   
**x_initial:** sampled space points at initial time   
**nSim_t:** number of (interior) time points sampled (size of t)  
**alpha:** mean of normal distribution for process staring value  
**beta:** standard deviation of normal distribution for process starting value  


In [8]:
# Loss function for Fokker-Planck equation

def loss(model, t, x_interior, x_initial, nSim_t, alpha, beta):
    
    # Loss term1: PDE
    
    # initialize vector of losses
    losses_u = []
    
    # for each simulated interior time point
    for tIndex in range(nSim_t):
        
        curr_t = t[tIndex]
        t_vector = curr_t * tf.ones_like(x_interior)
        
        u    = model.call(t_vector, x_interior)
        u_t  = tf.gradients(u, t_vector)[0]
        u_x  = tf.gradients(u, x_interior)[0]
        u_xx = tf.gradients(u_x, x_interior)[0]

        psi_denominator = tf.reduce_sum(tf.exp(-u))
        psi = tf.reduce_sum( u_t*tf.exp(-u) ) / psi_denominator

        # PDE differential operator
        diff_f = -u_t + kappa - kappa*(x_interior- theta)*u_x - 0.5*sigma**2*(-u_xx + u_x**2) + psi
        
        # compute L2-norm of differential operator and attach to vector of losses
        currLoss = tf.reduce_mean(tf.square(diff_f)) 
        losses_u.append(currLoss)
    
    # average losses across sample time points 
    L1 = tf.add_n(losses_u) / nSim_t
    
    # Loss term2: boundary condition
    # no boundary condition for this problem
    
    # Loss term3: initial condition
    # compute negative-exponential of neural network-implied pdf at t = 0 i.e. the u in p = e^[-u(t,x)] / c(t)
    fitted_pdf = model.call(0*tf.ones_like(x_initial), x_initial)
    
    target_pdf  = 0.5*(x_initial - alpha)**2 / (beta**2)
    
    # average L2 error for initial distribution
    L3 = tf.reduce_mean(tf.square(fitted_pdf - target_pdf))

    return L1, L3

##### input(time, space domain interior, space domain at initial time)

In [9]:
# Set up network
model = DGMNet(nodes_per_layer, num_layers, 1)

t_tnsr = tf.placeholder(tf.float32, [None,1])
x_interior_tnsr = tf.placeholder(tf.float32, [None,1])
x_initial_tnsr = tf.placeholder(tf.float32, [None,1])

# loss 
L1_tnsr, L3_tnsr = loss(model, t_tnsr, x_interior_tnsr, x_initial_tnsr, nSim_t, alpha, beta)
loss_tnsr = L1_tnsr + L3_tnsr

u = model.call(t_tnsr, x_interior_tnsr)
p_unnorm = tf.exp(-u)

# set optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_tnsr)

# initialize variables
init_op = tf.global_variables_initializer()

# open session
sess = tf.Session()
sess.run(init_op)

Instructions for updating:
Use the retry module or similar alternatives.


In [10]:
#Train network
for i in range(sampling_stages):
    
    # sample uniformly from the required regions
    t, x_interior, x_initial = sampler(nSim_t, nSim_x_interior, nSim_x_initial)
    
    for j in range(steps_per_sample):
        loss,L1,L3,_ = sess.run([loss_tnsr, L1_tnsr, L3_tnsr, optimizer],
                                feed_dict = {t_tnsr:t, x_interior_tnsr:x_interior, x_initial_tnsr:x_initial})
        
    print(loss, L1, L3, i)

6.647045 2.4194834 4.2275615 0
6.266523 2.2147923 4.0517306 1
4.1185074 1.4072833 2.7112243 2
4.430644 1.2149463 3.215698 3
2.2074668 0.4990591 1.7084078 4
1.7181611 0.37702793 1.3411331 5
3.3938904 2.0650299 1.3288606 6
2.4269183 1.030864 1.3960543 7
0.8449986 0.3127013 0.53229725 8
0.9292029 0.29220816 0.6369947 9
0.4122085 0.1308738 0.2813347 10
0.316097 0.09268576 0.22341123 11
0.25118518 0.07323962 0.17794557 12
0.11736485 0.040881068 0.076483786 13
5.8022327 4.1627855 1.6394472 14
1.7594676 0.70033497 1.0591327 15
1.3304719 0.41092768 0.91954416 16
1.9862294 1.1211528 0.86507666 17
1.4386858 0.31993982 1.1187459 18
0.72758394 0.2825614 0.44502255 19
1.3302073 0.7189587 0.6112487 20
0.58294755 0.20887528 0.37407225 21
0.17020497 0.10647034 0.06373463 22
0.4978568 0.2836935 0.2141633 23
0.1518138 0.047531568 0.10428224 24
0.5037191 0.32729912 0.17641999 25
0.35061917 0.08889363 0.26172554 26
0.35403 0.105500184 0.24852984 27
0.64455736 0.2920912 0.35246614 28
0.4488389 0.25552127 0

0.07136992 0.057190634 0.01417929 221
0.10283323 0.08636024 0.016472984 222
0.053320646 0.04487452 0.008446125 223
0.040007506 0.025583839 0.014423666 224
0.064950116 0.04467041 0.020279704 225
0.019269135 0.009818623 0.009450512 226
0.054114476 0.038662355 0.0154521195 227
0.030195452 0.016126124 0.014069328 228
0.017425193 0.012550406 0.0048747864 229
0.030964898 0.021115003 0.009849895 230
0.03490246 0.027840454 0.007062007 231
0.029330574 0.01696994 0.012360635 232
0.9498061 0.8355746 0.11423145 233
0.1441867 0.08147082 0.06271588 234
0.27628198 0.11349126 0.16279073 235
0.06391585 0.022881573 0.041034278 236
0.06642775 0.030380607 0.03604715 237
0.20345798 0.11610812 0.08734986 238
0.074724495 0.032682605 0.042041887 239
0.030007048 0.010545582 0.019461466 240
0.3674156 0.24914208 0.11827354 241
0.050655223 0.019096462 0.031558763 242
1.8746686 0.8179623 1.0567063 243
0.56444097 0.18447427 0.3799667 244
0.23637947 0.08199864 0.15438084 245
0.13570228 0.023838965 0.111863315 246
0.

0.036255047 0.014054889 0.02220016 430
0.26813513 0.20081814 0.06731699 431
0.10496172 0.020007337 0.08495439 432
0.05478204 0.007739454 0.047042586 433
0.1885358 0.09718714 0.091348656 434
0.22163376 0.17065488 0.050978877 435
0.066049404 0.05053724 0.015512163 436
0.091330275 0.05125503 0.040075246 437
0.04306897 0.023405222 0.01966375 438
0.030295307 0.0131288115 0.017166495 439
0.12468628 0.08045317 0.044233106 440
0.03211472 0.007230818 0.024883904 441
0.06849969 0.026002048 0.042497646 442
0.09537386 0.087226935 0.008146924 443
0.041550487 0.016846202 0.024704285 444
0.056221496 0.04146887 0.014752626 445
0.06224779 0.05381469 0.0084331 446
0.03920327 0.022786358 0.016416913 447
0.030551802 0.022222355 0.008329447 448
0.03560619 0.016932212 0.01867398 449
0.024729978 0.007860036 0.016869944 450
0.017388398 0.00657697 0.010811428 451
0.016184567 0.012405605 0.0037789615 452
0.44310716 0.38764828 0.05545887 453
0.18568614 0.15530221 0.030383935 454
0.039950095 0.012333092 0.0276170

In [11]:
# saver = tf.train.Saver()
# saver.save(sess, './FokkerPlack/' + saveName)